In [16]:
import os

# If you are using the FABRIC JupyterHub, the following three evnrionment vars
# were automatically provided when you logged in.
#os.environ['FABRIC_CREDMGR_HOST']='cm.fabric-testbed.net'
#os.environ['FABRIC_ORCHESTRATOR_HOST']='orchestrator.fabric-testbed.net'
#os.environ['FABRIC_TOKEN_LOCATION']=os.environ['HOME']+'/work/fabric_token.json'

# Set your FABRIC PROJECT ID
os.environ['FABRIC_PROJECT_ID']='6ce270de-788d-4e07-8bae-3206860a6387'

# Bastion IPs
os.environ['FABRIC_BASTION_HOST'] = 'bastion-1.fabric-testbed.net'

# Set your Bastion username and private key
os.environ['FABRIC_BASTION_USERNAME']='cvankaya_0000027070'
os.environ['FABRIC_BASTION_KEY_LOCATION']=os.environ['HOME']+'/work/fabric_config/bastion-key-cvankaya'

# Set the keypair FABRIC will install in your slice. 
os.environ['FABRIC_SLICE_PRIVATE_KEY_FILE']=os.environ['HOME']+'/.ssh/id_rsa'
os.environ['FABRIC_SLICE_PUBLIC_KEY_FILE']=os.environ['HOME']+'/.ssh/id_rsa.pub'

# If your slice private key uses a passphrase, set the passphrase
#from getpass import getpass
#print('Please input private key passphrase. Press enter for no passphrase.')
#os.environ['FABRIC_SLICE_PRIVATE_KEY_PASSPHRASE']=getpass()

In [17]:
import json
import traceback

from fabrictestbed_extensions.fablib.fablib import fablib

In [18]:
SLICE_NAME = 'p4-lb-1'

# We will use Ubuntu 20.04 for both nodes
IMAGE = 'default_ubuntu_20'
CORES = 1
RAM = 2
DISK_R = 50
DISK_H = 20
SITE="UCSD"

routers = {"r1":{}, "r2":{}}
lb = "lb"
clients = {"c1":{}, "c2":{}}
servers = {"s1":{}, "s2":{}}

In [19]:
import datetime
from datetime import timedelta

#Create Slice
slice = fablib.new_slice(name=SLICE_NAME)
now = datetime.datetime.now(datetime.timezone.utc)
end_date = (now + timedelta(days=6)).strftime("%Y-%m-%d %H:%M:%S %z")
#slice.renew(end_date)

#Create Nodes
#LB Node
lb_node = slice.add_node(name="lb", site=SITE, cores=CORES, ram=RAM, image=IMAGE, disk=DISK_R)

#Server Nodes
for s in servers.keys():
    servers[s]["node"] = slice.add_node(name = s, site=SITE, cores=CORES, ram=RAM, image=IMAGE, disk=DISK_H)

#Router Nodes
for r in routers.keys():
    routers[r]["node"] = slice.add_node(name = r, site=SITE, cores=CORES, ram=RAM, image=IMAGE, disk=DISK_R)

#Client Nodes
for c in clients.keys():
    clients[c]["node"] = slice.add_node(name = c, site=SITE, cores=CORES, ram=RAM, image=IMAGE, disk=DISK_H)

#Create Network
#LB-Servers
for s, v in servers.items():
    lb_iface = lb_node.add_component(model='NIC_Basic', name="if_{}".format(s)).get_interfaces()[0]
    s_iface = v["node"].add_component(model='NIC_Basic', name="if").get_interfaces()[0]
    net = slice.add_l2network(name="net_lb_{}".format(s), interfaces=[lb_iface, s_iface])

#LB-Routers
for r, v in routers.items():
    lb_iface = lb_node.add_component(model='NIC_Basic', name="if_{}".format(r)).get_interfaces()[0]
    r_iface = v["node"].add_component(model='NIC_Basic', name="if_lb").get_interfaces()[0]
    net = slice.add_l2network(name="net_lb_{}".format(r), interfaces=[lb_iface, r_iface])

#Todo Router-Router

#Routers-Clients
for r, c in zip(routers.keys(), clients.keys()):
    r_iface = routers[r]["node"].add_component(model='NIC_Basic', name="if_c").get_interfaces()[0]
    c_iface = clients[c]["node"].add_component(model='NIC_Basic', name="if").get_interfaces()[0]
    net = slice.add_l2network(name="net_{}_{}".format(r,c), interfaces=[r_iface, c_iface])

In [20]:
#Submit Slice Request
slice.submit()


-----------  ------------------------------------
Slice Name   p4-lb-1
Slice ID     8d7ef727-19fa-40d9-aad2-6e9835b980aa
Slice State  StableOK
Lease End    2022-11-29 02:52:13 +0000
-----------  ------------------------------------

Retry: 11, Time: 226 sec

ID                                    Name    Site    Host                          Cores    RAM    Disk  Image              Management IP    State    Error
------------------------------------  ------  ------  --------------------------  -------  -----  ------  -----------------  ---------------  -------  -------
854cefff-bc79-4364-bd6e-6c04a516c7c2  lb      UCSD    ucsd-w2.fabric-testbed.net        1      4     100  default_ubuntu_20  132.249.252.148  Active
bfcbb653-6a94-4426-885c-81fe5954a339  s1      UCSD    ucsd-w2.fabric-testbed.net        1      4     100  default_ubuntu_20  132.249.252.139  Active
6cac9afc-ce8f-496b-b1fd-c715a462e6ee  s2      UCSD    ucsd-w2.fabric-testbed.net        1      4     100  default_ubuntu_20  1

'8d7ef727-19fa-40d9-aad2-6e9835b980aa'

In [22]:
now = datetime.datetime.now(datetime.timezone.utc)
end_date = (now + timedelta(days=14)).strftime("%Y-%m-%d %H:%M:%S %z")
slice.renew(end_date)

In [23]:
#Create Subnets
from ipaddress import ip_address, IPv4Address, IPv6Address, IPv4Network, IPv6Network

try:
    subnet1 = IPv4Network("192.168.1.0/24")
    ips1 = list(subnet1)[1:]
    subnet2 = IPv4Network("192.168.2.0/24")
    ips2 = list(subnet2)[1:]
    subnet3 = IPv4Network("192.168.3.0/24")
    ips3 = list(subnet3)[1:]
except Exception as e:
    print(f"Exception: {e}")

In [24]:
#Assign IPs
lb_ips = {}
#LB-Servers
for s in servers.keys():
    node = slice.get_node("lb")
    iface = node.get_interface(network_name="net_lb_{}".format(s)) 
    ip = ips3.pop(0)
    print(ip)
    iface.ip_addr_add(addr=ip, subnet=subnet3)
    lb_ips[s]=ip
    print("Configuring lb IP {} for {}".format(ip, s))
    stdout, stderr = node.execute(f'ip addr show {iface.get_os_interface()}')
    print (stdout)

192.168.3.1
Configuring lb IP 192.168.3.1 for s1
5: ens9: <BROADCAST,MULTICAST,UP,LOWER_UP> mtu 1500 qdisc mq state UP group default qlen 1000
    link/ether 1a:84:1f:2d:6b:87 brd ff:ff:ff:ff:ff:ff
    inet 192.168.3.1/24 scope global ens9
       valid_lft forever preferred_lft forever
    inet6 fe80::1884:1fff:fe2d:6b87/64 scope link 
       valid_lft forever preferred_lft forever

192.168.3.2
Configuring lb IP 192.168.3.2 for s2
4: ens8: <BROADCAST,MULTICAST,UP,LOWER_UP> mtu 1500 qdisc mq state UP group default qlen 1000
    link/ether 1a:52:a9:bb:94:c0 brd ff:ff:ff:ff:ff:ff
    inet 192.168.3.2/24 scope global ens8
       valid_lft forever preferred_lft forever
    inet6 fe80::1852:a9ff:febb:94c0/64 scope link 
       valid_lft forever preferred_lft forever



In [25]:
#LB-Routers
#R1
node = slice.get_node("lb")
iface = node.get_interface(network_name="net_lb_r1") 
ip = ips1.pop(0)
print(ip)
iface.ip_addr_add(addr=ip, subnet=subnet1)
lb_ips["r1"]=ip
print("Configuring lb IP {} for {}".format(ip, "r1"))
stdout, stderr = node.execute(f'ip addr show {iface.get_os_interface()}')
print (stdout)

#R2
node = slice.get_node("lb")
iface = node.get_interface(network_name="net_lb_r2") 
ip = ips2.pop(0)
print(ip)
iface.ip_addr_add(addr=ip, subnet=subnet2)
lb_ips["r2"]=ip
print("Configuring lb IP {} for {}".format(ip, "r2"))
stdout, stderr = node.execute(f'ip addr show {iface.get_os_interface()}')
print (stdout)

192.168.1.1
Configuring lb IP 192.168.1.1 for r1
6: ens10: <BROADCAST,MULTICAST,UP,LOWER_UP> mtu 1500 qdisc mq state UP group default qlen 1000
    link/ether 1a:ed:2c:a5:7d:13 brd ff:ff:ff:ff:ff:ff
    inet 192.168.1.1/24 scope global ens10
       valid_lft forever preferred_lft forever
    inet6 fe80::18ed:2cff:fea5:7d13/64 scope link 
       valid_lft forever preferred_lft forever

192.168.2.1
Configuring lb IP 192.168.2.1 for r2
3: ens7: <BROADCAST,MULTICAST,UP,LOWER_UP> mtu 1500 qdisc mq state UP group default qlen 1000
    link/ether 02:73:45:58:72:b0 brd ff:ff:ff:ff:ff:ff
    inet 192.168.2.1/24 scope global ens7
       valid_lft forever preferred_lft forever
    inet6 fe80::73:45ff:fe58:72b0/64 scope link 
       valid_lft forever preferred_lft forever



In [26]:
#Servers
for s in servers.keys():
    node = slice.get_node(s)
    iface = node.get_interface(network_name="net_lb_{}".format(s)) 
    ip = ips3.pop(0)
    print(ip)
    iface.ip_addr_add(addr=ip, subnet=subnet3)
    servers[s]["ip"]=ip
    print("Configuring lb IP {} for {}".format(ip, s))
    stdout, stderr = node.execute(f'ip addr show {iface.get_os_interface()}')
    print (stdout)

192.168.3.3
Configuring lb IP 192.168.3.3 for s1
3: ens7: <BROADCAST,MULTICAST,UP,LOWER_UP> mtu 1500 qdisc mq state UP group default qlen 1000
    link/ether 1e:87:a0:66:ee:73 brd ff:ff:ff:ff:ff:ff
    inet 192.168.3.3/24 scope global ens7
       valid_lft forever preferred_lft forever
    inet6 fe80::1c87:a0ff:fe66:ee73/64 scope link 
       valid_lft forever preferred_lft forever

192.168.3.4
Configuring lb IP 192.168.3.4 for s2
3: ens7: <BROADCAST,MULTICAST,UP,LOWER_UP> mtu 1500 qdisc mq state UP group default qlen 1000
    link/ether 1e:c2:9a:90:9d:8d brd ff:ff:ff:ff:ff:ff
    inet 192.168.3.4/24 scope global ens7
       valid_lft forever preferred_lft forever
    inet6 fe80::1cc2:9aff:fe90:9d8d/64 scope link 
       valid_lft forever preferred_lft forever



In [27]:
#Routers-Clients

for r, c in zip(routers.keys(), clients.keys()):
    #Router-Client
    node = slice.get_node(r)
    iface = node.get_interface(network_name="net_{}_{}".format(r,c))
    if r == 'r1':
        ips = ips1
        subnet = subnet1
    else:
        ips = ips2
        subnet = subnet2
    print(ips)
    
    ip = ips.pop(0)
    
    iface.ip_addr_add(addr=ip, subnet=subnet)
    routers[r]["ip"] = {c:ip}
    print("Configuring router IP {} for {}".format(ip, c))
    stdout, stderr = node.execute(f'ip addr show {iface.get_os_interface()}')
    print (stdout)

    #Router-LB
    iface = node.get_interface(network_name="net_lb_{}".format(r))
    ip = ips.pop(0)
    iface.ip_addr_add(addr=ip, subnet=subnet)
    routers[r]["ip"]["lb"] = ip
    print("Configuring router IP {} for lb".format(ip))
    stdout, stderr = node.execute(f'ip addr show {iface.get_os_interface()}')
    print (stdout)
    
    #Client
    node = slice.get_node(c)
    iface = node.get_interface(network_name="net_{}_{}".format(r,c)) 
    ip = ips.pop(0)
    print(ip)
    iface.ip_addr_add(addr=ip, subnet=subnet)
    clients[c]["ip"]=ip
    print("Configuring IP {} for {}".format(ip, c))
    stdout, stderr = node.execute(f'ip addr show {iface.get_os_interface()}')
    print (stdout)

[IPv4Address('192.168.1.2'), IPv4Address('192.168.1.3'), IPv4Address('192.168.1.4'), IPv4Address('192.168.1.5'), IPv4Address('192.168.1.6'), IPv4Address('192.168.1.7'), IPv4Address('192.168.1.8'), IPv4Address('192.168.1.9'), IPv4Address('192.168.1.10'), IPv4Address('192.168.1.11'), IPv4Address('192.168.1.12'), IPv4Address('192.168.1.13'), IPv4Address('192.168.1.14'), IPv4Address('192.168.1.15'), IPv4Address('192.168.1.16'), IPv4Address('192.168.1.17'), IPv4Address('192.168.1.18'), IPv4Address('192.168.1.19'), IPv4Address('192.168.1.20'), IPv4Address('192.168.1.21'), IPv4Address('192.168.1.22'), IPv4Address('192.168.1.23'), IPv4Address('192.168.1.24'), IPv4Address('192.168.1.25'), IPv4Address('192.168.1.26'), IPv4Address('192.168.1.27'), IPv4Address('192.168.1.28'), IPv4Address('192.168.1.29'), IPv4Address('192.168.1.30'), IPv4Address('192.168.1.31'), IPv4Address('192.168.1.32'), IPv4Address('192.168.1.33'), IPv4Address('192.168.1.34'), IPv4Address('192.168.1.35'), IPv4Address('192.168.

In [21]:
# Add routes

In [28]:
try:
    slice = fablib.get_slice(name=SLICE_NAME)
    slice.delete()
except Exception as e:
    print(f"Fail: {e}")
    traceback.print_exc()